# Student Name: Ferreira da Silva LUCAS
# Student No.: 2311328
# Laboratory: Optical Media Interface Lab

# Exercise 3.1: Basics of pivot calibration

## 1 - Create a set of functions to perform rotation and coordinate transformation of a point in 3D space.

In [1]:
# Lucas
# All imports go here

import numpy as np
import os

In [2]:
# Lucas - 2024/01/16
# Define functions to rotate a point given a rotataion_matrix and
# to transform a point (rotate + translate)
# based on: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html
def rotate_point(point, rotation_matrix):
    """
    Rotate a point in 3D space.
    
    """
    return np.dot(rotation_matrix, point)

def transform_point(point, rotation_matrix, translation_vector):
    """
    Apply rotation and translation to a point in 3D space.
    
    """
    return rotate_point(point, rotation_matrix) + translation_vector

## 2 - Find the position [Rk,tk] (Rk: 3×3 rotation matrix, tk: 3×1 translation vector) of the object coordinate system with respect to the camera coordinate system from 𝑨j,k (j-th LED marker measured at frame k).

In [3]:
# Lucas - 2024/01/16
# Firstly, read the marker data from the txt file provided
# Based on: https://hackernoon.com/how-to-read-text-file-in-python
def read_marker_data(file_path, num_leds_per_frame=4):
    """
    Read the marker data from the file and organize it into frames.

    """
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Skip header (1st line of txt)

    # Each line contains three coordinates separated by whitespace
    data = np.array([list(map(float, line.split())) for line in lines])
    
    return data.reshape(-1, num_leds_per_frame, 3)  # Reshape into [frame, marker, coordinate]

# Lucas - 2024/01/16
# Then, define the rotation matix and translation vector
# Based on: https://dfki-ric.github.io/pytransform3d/rotations.html
# and https://sigmoidal.ai/en/matrix-transformations-and-coordinate-systems-with-python/
def calculate_rk_tk(frames):
    """
    Calculate the rotation matrices and translation vectors for each frame.

    """
    rotation_matrices = []
    translation_vectors = []

    for frame in frames:
        p1, p2, p3, p4 = frame  # Extract points, they are aligned with xyz-axis

        # Calculate the basis vectors
        x_axis = p2 - p1
        y_axis = p3 - p1
        z_axis = p4 - p1

        # Normalize the vectors to define the ortonormal basis
        # based on: https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html
        x_axis /= np.linalg.norm(x_axis)
        y_axis /= np.linalg.norm(y_axis)
        z_axis /= np.linalg.norm(z_axis)

        # Form the rotation matrix
        rotation_matrix = np.column_stack([x_axis, y_axis, z_axis])
        rotation_matrices.append(rotation_matrix)

        # Translation vector is the position of P1
        translation_vectors.append(p1)

    return rotation_matrices, translation_vectors

In [4]:
# Lucas - 2024/01/16
# Load data
file_path = '../Assignment3/pa1-1-tracker-05.txt'
frames = read_marker_data(file_path)
# Get rotation and translation -> Rk and tk!!!
rotation_matrices, translation_vectors = calculate_rk_tk(frames)

# Test an example point in the object coordinate system
point_in_object_coords = np.array([100, 0, 0])
transformed_point = transform_point(point_in_object_coords, rotation_matrices[0], translation_vectors[0])

print("Transformed Point in Camera Coordinate System:", transformed_point)

Transformed Point in Camera Coordinate System: [507.48205231 419.78306377 196.23776899]


## 3 - Find the pointer tip position btip on the object coordinate system by using the following least-squares method. 

In [5]:
# Lucas - 2024/01/16
# Define the operation to solve the linear system using least squared method
# Based on: 
def solve_for_b_tip_and_b_post(rotation_matrices, translation_vectors):
    """
    Solve for the pointer tip position and the post postition in the object coordinate 
    system using least squares.

    """
    num_frames = len(rotation_matrices)
    A = np.empty((0, 6))  # 6 = 3 for b_tip + 3 for b_post
    b = np.empty((0, 1))
    
    for k in range(num_frames):
        # Construct the problem matrix formulation as instructed in the assignemtn
        # [R -I] [b_tip b_post] =~ [-t]
        Rk = rotation_matrices[k]
        tk = translation_vectors[k]
        A_k = np.hstack([Rk, -np.eye(3)]) # Stack the rotation and - Identity
        b_k = -tk.reshape((3, 1))  # Convert to column vector
        A = np.vstack([A, A_k])
        b = np.vstack([b, b_k])
  
    # Solving the least squares problem
    # Based on: https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    b_tip = x[:3]  # The first three elements correspond to b_tip
    b_post = x[3:]  # The last three elements correspond to b_post

    return b_tip, b_post

In [6]:
# Lucas - 2024/01/16
# Get the estimation for b_tip
b_tip, b_post = solve_for_b_tip_and_b_post(rotation_matrices, translation_vectors)
print("Pointer Tip Position in Object Coordinate System:", b_tip.ravel())
print("Post position:", b_post.ravel())

Pointer Tip Position in Object Coordinate System: [  50.00000015   50.00000013 -199.99999974]
Post position: [5.00000000e+02 5.00000000e+02 2.74105102e-07]


## 4 - Apply the same algorithm on all the given measurement data files, and check the estimation error of btip when Aj,k contains measurement errors (0.1 mm and 1.0 mm) or when the pivot angle (the range of angles at which the pointer probe is moved) varied.

In [7]:
# Lucas - 2024/01/16
# Ground_truth_b_tip is known and provided in file pa1-1-true-tip-post.txt
ground_truth_b_tip = np.array([[50], [50], [-200]])  # Replace with actual ground truth values

# List of files and their respective pivot angles, measurement error and num of leds
files_with_info = [
    {'file_path': '../Assignment3/pa1-1-tracker-30.txt', 'pivot_angle': 30, 'measurement_error': '0mm', 'num_leds': 4},
    {'file_path': '../Assignment3/pa1-1-tracker-05.txt', 'pivot_angle': 5, 'measurement_error': '0mm', 'num_leds': 4},
    {'file_path': '../Assignment3/pa1-1-tracker-30-noisy1.txt', 'pivot_angle': 30, 'measurement_error': '0.1mm', 'num_leds': 4},
    {'file_path': '../Assignment3/pa1-1-tracker-05-noisy1.txt', 'pivot_angle': 5, 'measurement_error': '0.1mm', 'num_leds': 4},
    {'file_path': '../Assignment3/pa1-1-tracker-30-noisy2.txt', 'pivot_angle': 30, 'measurement_error': '1mm', 'num_leds': 4},
    {'file_path': '../Assignment3/pa1-1-tracker-05-noisy2.txt', 'pivot_angle': 5, 'measurement_error': '1mm', 'num_leds': 4},
]

# Lucas - 2024/01/16
# Iterate over the files and perform the calculation of b_tip (estimate)
for file_info in files_with_info:
    file_path = file_info['file_path']
    pivot_angle = file_info['pivot_angle']
    measurement_error = file_info['measurement_error']
    num_leds = file_info['num_leds']
    
    if os.path.exists(file_path):
        # Read the data from the file
        frames = read_marker_data(file_path)

        # Calculate rotation matrices and translation vectors
        rotation_matrices, translation_vectors = calculate_rk_tk(frames)
        
        # Solve for b_tip and b_post
        b_tip, b_post = solve_for_b_tip_and_b_post(rotation_matrices, translation_vectors)
        
        # Calculate the estimation error
        error = np.linalg.norm(b_tip - ground_truth_b_tip)

        print(f"Estimation error for {file_path} with pivot angle {pivot_angle}° / {measurement_error} / {num_leds} leds: {error}")
    else:
        print(f"File {file_path} does not exist.")

Estimation error for ../Assignment3/pa1-1-tracker-30.txt with pivot angle 30° / 0mm / 4 leds: 3.718414880344321e-07
Estimation error for ../Assignment3/pa1-1-tracker-05.txt with pivot angle 5° / 0mm / 4 leds: 3.245652591557124e-07
Estimation error for ../Assignment3/pa1-1-tracker-30-noisy1.txt with pivot angle 30° / 0.1mm / 4 leds: 0.07695651849335716
Estimation error for ../Assignment3/pa1-1-tracker-05-noisy1.txt with pivot angle 5° / 0.1mm / 4 leds: 0.3967156171183092
Estimation error for ../Assignment3/pa1-1-tracker-30-noisy2.txt with pivot angle 30° / 1mm / 4 leds: 0.33378009921156115
Estimation error for ../Assignment3/pa1-1-tracker-05-noisy2.txt with pivot angle 5° / 1mm / 4 leds: 6.626655297117109


# Exercise 3.2: Practical pivot calibration

## 1 - Implement and algorithm for aligning 3D point cloud with known correspondence. 

In [8]:
# Lucas - 2024/01/16
# Read the pj data, it is comma separated (different from previous loaded data - spaced separated)
def read_pj_data(file_path):
    """
    Read the LED marker positions (Pj) data from the file.

    """
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # skip header
    
    # Each line has the coords separated by comma
    data = np.array([list(map(float, line.split(','))) for line in lines])
    return data

# Lucas - 2024/01/16
# Compute quaternion method to align two sets of data
# Also returns the rotation and translation vector
# Uses SVD
# Based on: https://en.wikipedia.org/wiki/Point-set_registration and
# https://www.cse.wustl.edu/~taoju/cse554/lectures/lect07_Alignment.pdf
def align_point_sets_and_get_transformation(pj_data, aj_data):
    """
    Align two sets of 3D points and get the transformation matrix and translation vector.

    Args:
        pj_data : Array of points in the object coordinate system
        aj_data : Array of points in the camera coordinate system.

    """
    
    # Lucas - 2024/01/16
    # Centering the data using centroids
    centroid_pj = np.mean(pj_data, axis=0)
    centroid_aj = np.mean(aj_data, axis=0)
    pj_centered = pj_data - centroid_pj
    aj_centered = aj_data - centroid_aj

    # Constructing the correlation matrix
    H = np.dot(pj_centered.T, aj_centered)

    # Determining the optimal rotation
    # Based on: https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html
    # https://www.askpython.com/python/examples/singular-value-decomposition
    U, _, Vt = np.linalg.svd(H)
    R_optimal = np.dot(Vt.T, U.T)
    
    # Based on: https://numpy.org/doc/stable/reference/generated/numpy.linalg.det.html
    # https://www.cse.wustl.edu/~taoju/cse554/lectures/lect07_Alignment.pdf
    if np.linalg.det(R_optimal) < 0:
        Vt[-1, :] *= -1
        R_optimal = np.dot(Vt.T, U.T)

    # Applying the rotation
    pj_aligned = np.dot(pj_centered, R_optimal.T) + centroid_aj
    
    # Translation vector is the difference in the centroids
    t_vector = centroid_aj - np.dot(centroid_pj, R_optimal.T)
    
    return pj_aligned, R_optimal, t_vector

## 2. Calculate the position [Rk, tk] (i.e., object coordinate system with respect to the camera coordinate system) by aligning the positions (A1, A2, ..., AN) and (P1, P2, ..., PN) (i.e., find the coordinate transformation [R, t] that satisfy Aj = [[R  t] [0 0]] . Pj for all j).

In [9]:
# Lucas - 2024/01/16
# Now:
    # 1 - go over the files
    # 2 - align aj and pj, recovering Rk and tk
    # 3 - use R and t to estimate the tip position
    # 4 - compute the error given the ground truth value

# List of files and their respective pivot angles, measuremnt error and num of leds
files_with_info = [
    {'file_path': '../Assignment3/pa1-2-tracker-30.txt', 'pivot_angle': 30, 'measurement_error': '0mm', 'num_leds': 20},
    {'file_path': '../Assignment3/pa1-2-tracker-05.txt', 'pivot_angle': 5, 'measurement_error': '0mm', 'num_leds': 20},
    {'file_path': '../Assignment3/pa1-2-tracker-30-noisy1.txt', 'pivot_angle': 30, 'measurement_error': '0.1mm', 'num_leds': 20},
    {'file_path': '../Assignment3/pa1-2-tracker-05-noisy1.txt', 'pivot_angle': 5, 'measurement_error': '0.1mm', 'num_leds': 20},
    {'file_path': '../Assignment3/pa1-2-tracker-30-noisy2.txt', 'pivot_angle': 30, 'measurement_error': '1mm', 'num_leds': 20},
    {'file_path': '../Assignment3/pa1-2-tracker-05-noisy2.txt', 'pivot_angle': 5, 'measurement_error': '1mm', 'num_leds': 20},
]

# Iterate over files and compute the error
for file_info in files_with_info:
    aj_data_path = file_info['file_path']
    pivot_angle = file_info['pivot_angle']
    measurement_error = file_info['measurement_error']
    num_leds = file_info['num_leds']
    
    if os.path.exists(file_path):
    
        pj_data_path = '../Assignment3/pa1-2-pointer-probe.txt' # static data, provided in txt file
        pj_data = read_pj_data(pj_data_path)
        
        aj_data = read_marker_data(aj_data_path, num_leds_per_frame=20) # each of the reference txt files
    
        ground_truth_b_tip = np.array([[0], [0], [-200]]) # provided in txt file
    
        # Align each frame in aj_data with pj_data
        pj_aligned_list, R_list, t_list = [], [], []
        for aj_frame in aj_data:
            pj_aligned, R, t = align_point_sets_and_get_transformation(pj_data, aj_frame) # pj_data is static
            pj_aligned_list.append(pj_aligned)
            R_list.append(R)
            t_list.append(t)
        
        # estimate the tip position
        estimated_b_tip, _ = solve_for_b_tip_and_b_post(R_list, t_list)

        # compute the error
        # based on: https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html
        error = np.linalg.norm(estimated_b_tip - ground_truth_b_tip)
        print(f"Estimation error for {aj_data_path} with pivot angle {pivot_angle}° / {measurement_error} / {num_leds} leds: {error}")
    
    else:
        print(f"File {file_path} does not exist.")

Estimation error for ../Assignment3/pa1-2-tracker-30.txt with pivot angle 30° / 0mm / 20 leds: 4.3088763400644297e-07
Estimation error for ../Assignment3/pa1-2-tracker-05.txt with pivot angle 5° / 0mm / 20 leds: 3.7046361133282515e-07
Estimation error for ../Assignment3/pa1-2-tracker-30-noisy1.txt with pivot angle 30° / 0.1mm / 20 leds: 0.030385054399893682
Estimation error for ../Assignment3/pa1-2-tracker-05-noisy1.txt with pivot angle 5° / 0.1mm / 20 leds: 0.013322634170980454
Estimation error for ../Assignment3/pa1-2-tracker-30-noisy2.txt with pivot angle 30° / 1mm / 20 leds: 0.21969493929721903
Estimation error for ../Assignment3/pa1-2-tracker-05-noisy2.txt with pivot angle 5° / 1mm / 20 leds: 0.3826045488251913
